In [0]:
from datetime import date
import sys,re
from cryptography.fernet import Fernet
import json
import pandas as pd 
#typing_extensions

import os,itertools
from pathlib import Path
import snowflake.connector
from snowflake.snowpark.functions import *
from snowflake.snowpark import *
from snowflake.snowpark import DataFrame
from snowflake.snowpark.session import *
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import os
import re
#import helper as hp

from datetime import datetime, timedelta
from functools import reduce 
from snowflake.snowpark import Session
from snowflake.snowpark.functions import substr,col, when,substr, substring, udf, upper, lit, split, trim, expr, replace, is_null, approx_count_distinct,regexp_replace, max as snow_max,concat,to_date,row_number,  array_slice,array_size,get,charindex,length,startswith,contains,cast, radians, sin, cos, sqrt, atan2,sum,round,count#,union
from snowflake.snowpark.types import BooleanType, StringType, IntegerType, StructType, StructField, FloatType
import psutil
from snowflake.snowpark.exceptions import SnowparkSQLException
from snowflake.snowpark.window import Window
from snowflake.snowpark import functions as F
import math
import numpy as np

%run "./utility"

def compare_column_dtypes(df1, df2):
    dtypes_df1 = df1.dtypes
    dtypes_df2 = df2.dtypes
    dtype_comparison = pd.DataFrame({
        'Column': dtypes_df1.index,
        'df1_dtype': dtypes_df1.values,
        'df2_dtype': dtypes_df2.reindex(dtypes_df1.index).values
    })
    dtype_differences = dtype_comparison[dtype_comparison['df1_dtype'] != dtype_comparison['df2_dtype']]

    return dtype_differences


def convert_column_names_to_uppercase(df):
    df.columns = [col.upper() for col in df.columns]
    return df



def float_comparator(x,y):
    if np.isnan(x) and np.isnan(y):
        return True
    elif x ==y:
        return True
    else:
        return False

 
def comparator(x,y):
    if type(x) ==float and type(y)==float:
        return float_comparator(x,y)
    elif x is None and y is None:
        return True
    elif x ==y:
        return True
    else:
        return odd_compare(x,y)
def odd_compare_float(x,y): 
    if type(x)==float and np.isnan(x):
        if( str.strip(y)==''):
           return True
        else:
            return False
    else:
        if( str.strip(x)==''):
           return True
        else:
            return False     
def odd_compare(x,y):
    if type(x)==float or type(y)==float:
        return odd_compare_float(x,y)
    elif type(x)==str and type(y) ==str:
        return (True if str.strip(x)==str.strip(y) else False)
    else:
        return False


# returns True is same else False
def compare_dataframes(df1, df2, key_column, value_columns):
    value_columns = [x for x in value_columns if x not in key_columns]
    merged_df = df1.merge(df2, on=key_column, suffixes=('_df1', '_df2'))
    merged_df_results = merged_df
    #merged_df_keys = merged_df_results[key_columns]
    #print(merged_df.columns)
    #print('original count',df1.count,df2.count) #print("shapes of dfs",df1.shape,df2.shape)
    print("common records on given key",merged_df.shape)
    
    discrepancies = pd.DataFrame()

    for column in value_columns:
        merged_df_results[f'{column}_df1df2'] = merged_df_results.apply(lambda row: 1 if comparator(row[f'{column}_df1'],row[f'{column}_df2']) else 0, axis=1)

    return merged_df,merged_df_results#,merged_df_keys



def get_columns_with_max_diff(df):
    df_columns = df.columns
    columns_df1df2 = []
    for i in range(0,len(df_columns)):
        if df_columns[i].endswith('_df1df2'):
            columns_df1df2

def count_zeros_in_columns(df):
    # Identify columns ending with 'df1df2'
    target_columns = [col for col in df.columns if col.endswith('df1df2')]
    zero_counts = {}
    for col in target_columns:
        zero_counts[col] = (df[col] == 0).sum()
    sorted_zero_counts = dict(sorted(zero_counts.items(), key=lambda item: item[1], reverse=True))
    return zero_counts,sorted_zero_counts




def df_dtype_compare(df1,df2):
    dtypes_dict1 = df1.dtypes.apply(lambda x: x.name).to_dict()
    dtypes_dict2 = df2.dtypes.apply(lambda x: x.name).to_dict()    
    column_list1 = df1.columns
    column_list2 = df2.columns
    for i in range(0,len(column_list1)):
        col_i = column_list1[i]
        dtype1 = dtypes_dict1[col_i]
        dtype2 = dtypes_dict2[col_i]
        if(dtype1!=dtype2):
            print(col_i,dtype1,dtype2)        
def aligndataframes(df1,df2):
    dtypes_dict1 = df1.dtypes.apply(lambda x: x.name).to_dict()
    dtypes_dict2 = df2.dtypes.apply(lambda x: x.name).to_dict()
    column_list1 = df1.columns
    column_list2 = df2.columns
    for i in range(0,len(column_list1)):
        col_i = column_list1[i]
        dtype1 = dtypes_dict1[col_i]
        dtype2 = dtypes_dict2[col_i]
        try:
         if(dtype1!=dtype2):
             if(dtype2 == 'object' and ((dtype1 == 'float64') or (dtype1 == 'int64'))):
                df2[col_i] = df2[col_i].apply(lambda x:nan if str.strip (str(x)) =='' else x)
                df2[col_i] = df2[col_i].astype(df1[col_i].dtype)
             elif(((dtype2 == 'float64') or (dtype2 == 'int64')) and dtype1 == 'object'):
                 df1[col_i] = df1[col_i].apply(lambda x:nan if str.strip (str(x)) =='' else x)
                 df1[col_i] = df1[col_i].astype(df2[col_i].dtype)
             elif(dtype1 =='float64' and dtype2 =='int64'):
                 df2[col_i] = df2[col_i].astype(df1[col_i].dtype)     
             elif(dtype1 =='int64' and dtype2 =='float64'):
                 df1[col_i] = df1[col_i].astype(df2[col_i].dtype)             
             else:
                 print(dtype1,dtype2)
        except Exception as e:
            print(col_i,dtype1,dtype2)
            if(dtype2 == 'object' and ((dtype1 == 'float64') or (dtype1 == 'int64'))):
                df1[col_i] = df1[col_i].astype(df2[col_i].dtype)
                pd.set_option('future.no_silent_downcasting', True)
                df1[col_i] = df1[col_i].fillna('')
            elif(((dtype2 == 'float64') or (dtype2 == 'int64')) and dtype1 == 'object'):
                 df2[col_i] = df2[col_i].astype(df1[col_i].dtype)
                 pd.set_option('future.no_silent_downcasting', True)
                 df2[col_i] = df2[col_i].fillna('')
            elif(dtype1 =='float64' and dtype2 =='int64'):
                 df1[col_i] = df1[col_i].astype(df2[col_i].dtype)  
            else:
                 print(dtype1,dtype2)
    return df1,df2

def get_distinct_column_values(df, column_name):
    return df[column_name].unique()

def check_duplicates(df1,key_columns):
    df2 = df1.drop_duplicates(key_columns)
    print(len(df2),len(df1))
    print('difference: ',len(df2)-len(df1))




def compare_files_from_containers(blob1,blob2,container1,container2,connect_storage_str,key_columns,value_columns,sep1, lineterminator1):
    df1_data = get_filedata_from_container(connect_storage_str,container1,blob1)
    df2_data = get_filedata_from_container(connect_storage_str,container2,blob2)
    df1 = pd.read_csv(df1_data, sep=sep1, lineterminator=lineterminator1)
    df1 = convert_column_names_to_uppercase(df1)
    dtypes_dict = df1.dtypes.apply(lambda x: x.name).to_dict()
    df2 = pd.read_csv(df2_data, sep=sep1, lineterminator=lineterminator1)
    df2 = convert_column_names_to_uppercase(df2)
    df2['CC_ENTITY'] = df2['CC_ENTITY'].fillna(' ')
    print("shapes of dfs",df1.shape,df2.shape)
    df1,df2 = aligndataframes(df1,df2)
    print("difference records ",df1.shape[0]-df2.shape[0])
    #print(compare_column_dtypes(df1,df2))
    #return df1,df2
    merged_df,merged_df_results = compare_dataframes(df1, df2, key_columns, value_columns)
    return df1,df2,merged_df,merged_df_results

def get_df_before_compare_files_from_containers(blob1,blob2,container1,container2,connect_storage_str,sep1, lineterminator1):
    df1_data = get_filedata_from_container(connect_storage_str,container1,blob1)
    df2_data = get_filedata_from_container(connect_storage_str,container2,blob2)
    df1 = pd.read_csv(df1_data, sep=sep1, lineterminator=lineterminator1)
    df1 = convert_column_names_to_uppercase(df1)
    dtypes_dict = df1.dtypes.apply(lambda x: x.name).to_dict()
    df2 = pd.read_csv(df2_data, sep=sep1, lineterminator=lineterminator1)
    df2 = convert_column_names_to_uppercase(df2)
    #df2['CC_ENTITY'] = df2['CC_ENTITY'].fillna(' ')
    print("shapes of dfs",df1.shape,df2.shape)
    try:
       df1,df2 = aligndataframes(df1,df2)
    except Exception as e:
        print("error: ",e)
        return df1,df2
    print("difference records ",df1.shape[0]-df2.shape[0])
    #print(compare_column_dtypes(df1,df2))
    #return df1,df2
    return df1,df2

def get_df_before_compare_files_from_containers_dict(blob1,blob2,container1,container2,connect_storage_str,sep1, lineterminator1,columns_dict):
    try:
        df1_data = get_filedata_from_container(connect_storage_str,container1,blob1)
        df2_data = get_filedata_from_container(connect_storage_str,container2,blob2)
        df1 = pd.read_csv(df1_data, sep=sep1, lineterminator=lineterminator1, dtype=columns_dict)
        #df1 = pd.read_csv(df1_data, sep=sep1, lineterminator=lineterminator1)
        df1 = convert_column_names_to_uppercase(df1)
        df2 = pd.read_csv(df2_data, sep=sep1, lineterminator=lineterminator1, dtype=columns_dict)
        #df2 = pd.read_csv(df2_data, sep=sep1, lineterminator=lineterminator1)
        df2 = convert_column_names_to_uppercase(df2)
        #df2['CC_ENTITY'] = df2['CC_ENTITY'].fillna(' ')
        print("shapes of dfs",df1.shape,df2.shape) 
        #df1,df2 = aligndataframes(df1,df2)
    except Exception as e:
        print("error: ",e)
        df1 = pd.DataFrame()
        df2 = pd.DataFrame()
        return df1,df2
    print("difference records ",df1.shape[0]-df2.shape[0])
    #print(compare_column_dtypes(df1,df2))
    #return df1,df2
    return df1,df2






def find_unique_records_leftondf1(df1, df2, key_columns):
    merged_df = df1.merge(df2, on=key_columns, how='left', indicator=True)
    unique_records = merged_df[merged_df['_merge'] == 'left_only']
    unique_records = unique_records.drop(columns=['_merge'])
    return unique_records


def find_unique_records_inner(df1, df2, key_columns):
    merged_df = df1.merge(df2, on=key_columns, how='inner',suffixes =('_x','_y'))
    #order = get_column_order(df1.columns,key_columns)
    return merged_df#[order]

def compare_dataframes2(df1, df2, key_column, value_columns):
    value_columns = [x for x in value_columns if x not in key_columns]
    merged_df = df1.merge(df2, on=key_column, suffixes=('_df1', '_df2'))
    merged_df_results = merged_df
    merged_df_keys = merged_df_results[key_columns]
    #print(merged_df.columns)
    #print('original count',df1.count,df2.count) #print("shapes of dfs",df1.shape,df2.shape)
    print("common records on given key",merged_df.shape)
    discrepancies = pd.DataFrame()
    for column in value_columns:
        merged_df_results[f'{column}_df1df2'] = merged_df_results.apply(lambda row: 1 if comparator(row[f'{column}_df1'],row[f'{column}_df2']) else 0, axis=1)
    return merged_df,merged_df_results,merged_df_keys



def re_compare_files_from_containers(df1,df2,key_columns,value_columns):
    print("shapes of dfs",df1.shape,df2.shape)
    print("difference records ",df1.shape[0]-df2.shape[0])
    merged_df,merged_df_results,merged_df_keys = compare_dataframes2(df1, df2, key_columns, value_columns)
    return merged_df,merged_df_results#,merged_df_keys




def get_df3(df1, df2, key_columns):
    #merged_df = df2.merge(df1, on=key_columns, how='left', indicator=True, suffixes=('_df1', '_df2'))
    merged_df = df2.merge(df1, on=key_columns, how='left', indicator=True, suffixes=('_df1', '_df2'))
    df3 = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])
    df1_columns = df1.columns
    ordered_columns = key_columns + [col + suffix for col in df1.columns if col not in key_columns for suffix in ['_df1', '_df2']]
    df3 = df3[ordered_columns]
    return df3

def record_level_difference(key_columns, df1,df2):
    print("shapes of dfs",df1.shape,df2.shape)
    df1 = convert_column_names_to_uppercase(df1)
    df2 = convert_column_names_to_uppercase(df2)
    count1 = df1.shape[0]
    count2 = df2.shape[0]
    print('counts are:',count1,count2)
    print('coutn difference :', (count1-count2 if count1-count2>0 else count2-count1))
    if(count1>count2):
       print('df1 is bigger')
       #df3 = get_df3(df2, df1, key_columns)
    else:
        print('df2 is bigger')
        #df3 = get_df3(df1, df2, key_columns)
    df3 = get_df3(df1, df2, key_columns)
    df4 = get_df3(df2, df1, key_columns)
    print (df3.shape)
    print (df4.shape)
    return df3,df4


#key_columns = ['CLAIM_NUMBER','SERVICE_LINE']
#value_columns = ['SOURCE_SYSTEM','CC_entity','CLAIM_NUMBER','SERVICE_LINE','REVENUE_CODE','CLAIM_STATUS','FACILITY_TYPE_CODE','PROCEDURE_CODE','MODIFIER_1','MODIFIER_2','MODIFIER_3','Modifier_4','Emergency_Flag','NDC_Code','Hospital_Related_Flag','Outside_Labs_Flag','PAID_AMOUNT','Billed_Amount','Value_Amount','UNIT_COUNT','SERVICE_FROM_DATE','SERVICE_TO_DATE','COPAY_AMOUNT','DEDUCTIBLE_AMOUNT','COINSURANCE_AMOUNT','PATIENT_LIABILITY_AMOUNT','Adjustment_Code','Adjustment_Desc','Discount_Type_Code','DISCOUNT_AMOUNT','Adjudication_Date','Benefit_Limit_Amount','Admitting_Provider_ID','Business_Unit_Number','COVERED_AMOUNT','Earliest_Service_From_Date','Eligible_Amount','Last_Service_Date','National_Drug_Code','Net_Paid_Amount','NPI_TAXONOMY_CODE','Original_Place_of_Service_Code','Servicing_NPI_Taxonomy_Code','Subscriber_Number']
def to_uppercase_comprehension(strings):
    return [string.upper() for string in strings]
#key_columns = to_uppercase_comprehension(key_columns)




#to be used post compare_dataframes
def sort_objects_by_values(objects, values):
    paired_list = list(zip(objects, values))
    sorted_paired_list = sorted(paired_list, key=lambda x: x[1], reverse=False)
    sorted_objects = [item[0] for item in sorted_paired_list]
    return sorted_objects


def get_difference_records(md,value_columns,key_columns):
    value_columns = [x for x in value_columns if x not in key_columns]
    condition = False
    for i in range(0,len(value_columns)):
        condition = condition | (md[f'{value_columns[i]}_df1df2'] == 0)
    md = md[condition]
    df1df2_columns = []
    df1df2_columns_counts = []
    for  i in range(0,len(value_columns)):
        count = md[f'{value_columns[i]}_df1df2'].sum()
        df1df2_columns_counts.append(count)
    sorted_value_columns = sort_objects_by_values(value_columns,df1df2_columns_counts)
    final_column_list = key_columns
    for i in range(0,len(sorted_value_columns)):
        final_column_list.append(f'{sorted_value_columns[i]}_df1')
        final_column_list.append(f'{sorted_value_columns[i]}_df2')
        final_column_list.append(f'{sorted_value_columns[i]}_df1df2')
    md = md[final_column_list]
    return md



